In [ ]:
import menpo.io as mio
from menpo.visualize import print_progress
from menpo.landmark import labeller, face_ibug_68_to_face_ibug_68_trimesh

path_to_images = '/home/rshaydu/tensorfaces/helen/trainset/'
training_images = []
for img in print_progress(mio.import_images(path_to_images, verbose=True)):
    # convert to greyscale
    if img.n_channels == 3:
        img = img.as_greyscale()
    # crop to landmarks bounding box with an extra 20% padding
    img = img.crop_to_landmarks_proportion(0.2)
    # rescale image if its diagonal is bigger than 400 pixels
    d = img.diagonal()
    if d > 400:
        img = img.rescale(400.0 / d)
    # define a TriMesh which will be useful for Piecewise Affine Warp of HolisticAAM
    labeller(img, 'PTS', face_ibug_68_to_face_ibug_68_trimesh)
    # append to list
    training_images.append(img)



In [ ]:
%matplotlib inline
from menpowidgets import visualize_images
visualize_images(training_images)

In [ ]:
# from menpofit.aam import PatchAAM
# from menpo.feature import fast_dsift

# patch_aam = PatchAAM(training_images, group='PTS', patch_shape=[(15, 15), (23, 23)],
#                      diagonal=150, scales=(0.5, 1.0), holistic_features=fast_dsift,
#                      max_shape_components=20, max_appearance_components=150,
#                      verbose=True)
from menpofit.aam import HolisticAAM
from menpo.feature import fast_dsift

aam = HolisticAAM(training_images, group='face_ibug_68_trimesh', diagonal=150,
                  scales=(0.5, 1.0), holistic_features=fast_dsift, verbose=True,
                  max_shape_components=20, max_appearance_components=150)


In [ ]:
aam.view_appearance_models_widget()

In [ ]:
aam.view_aam_widget()

In [ ]:
from menpofit.aam import LucasKanadeAAMFitter, WibergInverseCompositional

fitter = LucasKanadeAAMFitter(aam, lk_algorithm_cls=WibergInverseCompositional,
                              n_shape=[5, 20], n_appearance=[30, 150])


In [ ]:
print(fitter)

In [ ]:
from pathlib import Path
import menpo.io as mio

path_to_lfpw = Path('/home/rshaydu/tensorfaces/helen/testset/')

image = mio.import_image(path_to_lfpw / '296814969_3.jpg')
image = image.as_greyscale()


In [ ]:
from menpodetect import load_opencv_frontal_face_detector

# Load detector
detect = load_opencv_frontal_face_detector()

# Detect
bboxes = detect(image)
print("{} detected faces.".format(len(bboxes)))

# View
if len(bboxes) > 0:
    image.view_landmarks(group='opencv_0', line_colour='red',
                         render_markers=False, line_width=4);

In [ ]:
# initial bbox
initial_bbox = bboxes[0]

# fit image
result = fitter.fit_from_bb(image, initial_bbox, max_iters=[15, 5],
                            gt_shape=image.landmarks['PTS'])

# print result
print(result)

In [ ]:
result.view(render_initial_shape=True)

In [ ]:
warped = fitter.warped_images(result.image, [result.final_shape])
warped[0].view()

In [ ]:
# from menpodetect import load_ffld2_frontal_face_detector

# # Load detector
# detect = load_ffld2_frontal_face_detector()

# # Detect
# bboxes = detect(image)
# print("{} detected faces.".format(len(bboxes)))

# # View
# if len(bboxes) > 0:
#     image.view_landmarks(group='llfd2_0', line_colour='red',
#                          render_markers=False, line_width=4);

In [ ]:
import matplotlib.pyplot as plt
from menpodetect import load_dlib_frontal_face_detector


# Load and convert to grayscale
image = mio.import_image('/home/rshaydu/tensorfaces/FaceBase_png/amir-vp0-il1-ex2.png')
image = image.as_greyscale()

# Load detector
detect = load_dlib_frontal_face_detector()

# # Detect face
bboxes = detect(image)
if len(bboxes) > 0:
    image.view_landmarks(group='dlib_0', line_colour='red',
                         render_markers=False, line_width=4);
else:
    print("Error: no faces detected")

In [ ]:
import numpy as np
import menpo

# initial bbox
# build box by hand
adjacency_matrix = np.array([[0,1,0,0],
                             [0,0,1,0],
                             [0,0,0,1],
                             [1,0,0,0]])
# points = np.array([[150,100],[500,100],[500,300],[150,300]]) # fine for looking right
points = np.array([[120,75],[450,75],[450,275],[120,275]])

initial_bbox = menpo.shape.PointDirectedGraph(points, adjacency_matrix)

# fit image
result = fitter.fit_from_bb(image, bboxes[0], max_iters=50)

# print result
print(result)


In [ ]:
result.view(render_initial_shape=True)

In [ ]:
result.view_iterations()
result.image.shape

In [ ]:
# result
# cropped = image.crop_to_pointcloud(result.final_shape)
# cropped.view()

In [ ]:
warped = fitter.warped_images(result.image, [result.final_shape, fitter.reference_shape])
warped[0].view()